In [11]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 현재 디렉토리의 피클 파일들을 불러오기
current_dir = '.'  # 현재 디렉토리
pickle_files = [f for f in os.listdir(current_dir) if f.endswith('.pkl')]  # .pkl 확장자 파일들
topk = [1,5,20]

In [12]:
def save_excel(df, dataframes_dict):
    # 엑셀 파일로 저장
    for file_name, sheet_dataframes in dataframes_dict.items():
        # 엑셀 파일 경로 설정 (파일명 변경)
        excel_path = os.path.splitext(file_name)[0] + '.xlsx'
        
        # 여러 시트를 가진 엑셀 파일로 저장
        with pd.ExcelWriter(excel_path) as writer:
            for i, df in enumerate(sheet_dataframes):
                sheet_name = f"topk_{topk[i]}"  # 시트 이름 설정
                df.to_excel(writer, sheet_name=sheet_name, index=False)  # 시트에 작성

    excel_path  # 마지막 파일의 저장 경로 확인

In [13]:
def load_pickle_and_convert_df(pickle_files):
    # 피클 파일들의 데이터를 데이터프레임으로 변환
    dataframes_dict = {}

    for file_name in pickle_files:
        file_path = os.path.join(current_dir, file_name)  # 피클 파일 경로
        
        # 피클 파일 불러오기
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        
        # 데이터가 리스트로 구성되어 있는지 확인 후 데이터프레임으로 변환
        sheet_dataframes = [pd.DataFrame(sheet_data) for sheet_data in data]  # 각 리스트를 데이터프레임으로 변환
        corrected_sheets = []
        
        for df in sheet_dataframes:
            # 각 행이 딕셔너리 형태인 경우 이를 펼쳐서 변환
            expanded_df = pd.json_normalize(df[0])  # 각 행의 딕셔너리를 개별 컬럼으로 변환
            corrected_sheets.append(expanded_df)
            thresholds = np.linspace(0.001, 0.8, 500)
            expanded_df.insert(0, 'Thresholds', thresholds[:len(expanded_df)])
        
        dataframes_dict[file_name] = corrected_sheets  # 파일 이름을 키로 데이터프레임 리스트 저장
    return dataframes_dict

In [24]:
dataframes_dict = load_pickle_and_convert_df(pickle_files)
for df in dataframes_dict.values():

    x = df[:, 0]  # 첫 번째 열
    y = df[:, 5]  # 여섯 번째 열
        
    # 그래프 생성
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, marker='o', linestyle='-')
    plt.xlabel(df.columns[0])  # 첫 번째 열의 이름을 x축 레이블로 사용
    plt.ylabel(df.columns[5])  # 여섯 번째 열의 이름을 y축 레이블로 사용
    # plt.title(f'{model_name}')
    plt.grid(True)
    plt.show()

TypeError: list indices must be integers or slices, not tuple